<img src="", width=150, ALIGN="left", border=20>
<center>
<h1>Air Quality Challenge Starting Kit</h1>
<br>This code was tested with <br>
Python 2.7.13 | Anaconda 4.3.1 (https://anaconda.org/)<br>
<a href="http://www.datascience-paris-saclay.fr">Paris Saclay Center for Data Science (CDS)</a>
</center>

ALL INFORMATION, SOFTWARE, DOCUMENTATION, AND DATA ARE PROVIDED "AS-IS". The CDS, CHALEARN, AND/OR OTHER ORGANIZERS OR CODE AUTHORS DISCLAIM ANY EXPRESSED OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR ANY PARTICULAR PURPOSE, AND THE WARRANTY OF NON-INFRIGEMENT OF ANY THIRD PARTY'S INTELLECTUAL PROPERTY RIGHTS. IN NO EVENT SHALL AUTHORS AND ORGANIZERS BE LIABLE FOR ANY SPECIAL, 
INDIRECT OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF SOFTWARE, DOCUMENTS, MATERIALS, PUBLICATIONS, OR INFORMATION MADE AVAILABLE FOR THE CHALLENGE. 

# Introduction

The Air Quality challenge is a regression problem inspired from the Kaggle challenge <a href="https://www.kaggle.com/nelsonchu/air-quality-in-northern-taiwan">Air Quality in Northern Taiwan</a>, formatted in the AutoML format. 


In [1]:
model_dir = 'sample_code_submission/'          
problem_dir = 'ingestion_program/'  
score_dir = 'scoring_program/'
from sys import path; path.append(model_dir); path.append(problem_dir); path.append(score_dir); 
%matplotlib inline
%load_ext autoreload
%autoreload 2
import seaborn as sns; sns.set()

<h1> Step 1: Exploratory data analysis </h1>
We provide sample_data with the starting kit, but to prepare your submission, you must fetch the public_data from the challenge website and point to it.

In [2]:
datadir = 'sample_data'              # Change this to the directory where you put the input data
dataname = 'air'
!ls $datadir*

air_feat.name	 air_test.data	    air_train.solution
air_label.name	 air_test.solution  air_valid.data
air_public.info  air_train.data     air_valid.solution


For convenience, we load the data as a "pandas" data frame, so we can use "pandas" and "seaborn" built in functions to explore the data. YOU MIGHT WANT TO KEEP THIS UNTIL THE NEXT YELLOW BLOCK.

In [3]:
from data_io import read_as_df
data = read_as_df(datadir  + '/' + dataname)                # The data are loaded as a Pandas Data Frame
target_name = data.columns.values[-1]                       # The last column is the target

ImportError: No module named yaml

In [ ]:
data.head()

In [ ]:
data.describe() 

In [ ]:
data.hist(figsize=(10, 10), bins=50, layout=(3, 5));

In [ ]:
for i in range(len(data.columns.values)-1):
    sns.jointplot(x="NOx", y=data.columns.values[i], data=data)

# Step 2: Building a predictive model


## Loading data with DataManager
We reload the data with the AutoML DataManager class because this is more convenient:

In [ ]:
from data_manager import DataManager
D = DataManager(dataname, datadir, replace_missing=True)
print D

In [ ]:
print(datadir)

## Training a predictive model
We provide an example of predictive model (for classification or regression) in the `sample_code_submission/` directory. It is a quite stupid model: it makes constant predictions. Replace it with your own model.

In [ ]:
from model import model
??model 

<h3>Create or reload model</h3>
Create an instance of the model (run the constructor) and attempt to reload a previously saved model from `sample_code_submission/*_model.pickle` if `reload_model=1`.<br>
If you saved have an aldeady trained model saved in sample_code_submission, the evaluation script will reload it and not retrain, just test. This will happen the second time you run the code because when the model is trained it gets saved. <br>
When you reload a model, you risk to reload a model trained on the wrong data. Delete `*_model.pickle` from `sample_code_submission/` if you do not want this to happen.

In [ ]:
M = model()   
reload_model=0                       # Change to 1 to reload an already trained model

trained_model_name = model_dir + dataname
if reload_model:    
    M = M.load(trained_model_name)                  # Attempts to re-load an already trained model

<h3> Training:</h3>

In [ ]:
X_train = D.data['X_train']
Y_train = D.data['Y_train']

if not(M.is_trained):                               # No need to train if model already trained
    M.fit(X_train, Y_train)  

### Making predictions:

In [ ]:
Y_hat_train = M.predict(D.data['X_train']) # Optional, not really needed to test on taining examples
Y_hat_valid = M.predict(D.data['X_valid'])
Y_hat_test = M.predict(D.data['X_test'])

### Saving:
Save the trained model (will be ready to reload next time around) and save the prediction results:

In [ ]:
M.save(trained_model_name)                 
result_name = 'sample_result_submission/' + dataname
from data_io import write
write(result_name + '_valid.predict', Y_hat_valid)
write(result_name + '_test.predict', Y_hat_test)
!ls $result_name*

## Scoring the results
### Load the challenge metric
<b>The metric chosen for your challenge</b> is identified in the "metric.txt" file found in the `scoring_function/` directory. We use here the `mse_metric` metric (an example of organizer-supplied metric found in `my_metric.py`), which computes the mean-square-error. You may change that in the "metric.txt" file to e.g. use `bac_multiclass`, one of the AutoML challenge metrics found in `libscores.py`, which is 2*(balanced_accuracy)-1.

In [ ]:
with open(score_dir + '/metric.txt', 'r') as f:
    metric_name = f.readline().strip()
import libscores, my_metric
try:
    scoring_function = getattr(libscores, metric_name)
except:
    scoring_function = getattr(my_metric, metric_name)
print 'Using scoring metric:', metric_name
??scoring_function

### Training performance
The participants normally posess target values (labels) only for training examples (except for the sample data). We compute with the `example` metric the training score, which should be zero for perfect predictions. 
<div style="background:#FFFFAA">
Isabelle: I found that the scoring program does not like when we compare a column vector and a line vector. This seems to be a problem only here.

In [ ]:
Y_train =  Y_train.ravel() # We need this conversion to a line vector

In [ ]:
print 'Training score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_hat_train)
print 'Ideal score for the', metric_name, 'metric = %5.4f' % scoring_function(Y_train, Y_train)

<div style="background:#FFFFAA">
<h4> Other metrics (optional):</h4>
You can here compare with other metrics.

In [ ]:
print 'Training score for the a_metric = %5.4f' % libscores.a_metric(Y_train, Y_hat_train)
print 'Ideal score for the a_metric = %5.4f' % libscores.a_metric(Y_train, Y_train)

<div style="background:#FFFFAA">
<H3> Cross-validation performance </H3>
The participants do not have access to the labels Y_valid and Y_test to self-assess their validation and test performances. But training performance is not a good prediction of validation or test performance. Using cross-validation, the training data is split into multiple training/test folds, which allows participants to self-assess their model during development. <br>
YOU MAY USE OTHER TYPES OF CV BUT KEEP THIS SECTION.


In [ ]:
from sklearn.model_selection import KFold
from numpy import zeros, mean
# 3-fold cross-validation
n = 3
kf = KFold(n_splits=n)
kf.get_n_splits(X_train)
i=0
scores = zeros(n)
for train_index, test_index in kf.split(X_train):
    Xtr, Xva = X_train[train_index], X_train[test_index]
    Ytr, Yva = Y_train[train_index], Y_train[test_index]
    M = model()
    M.fit(Xtr, Ytr)
    Yhat = M.predict(Xva)
    scores[i] = scoring_function(Yva, Yhat)
    print ('Fold', i+1, 'example metric = ', scores[i])
    i=i+1
print ('Average score = ', mean(scores))

# Step 3: Making a submission

## Unit testing

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. All you have to do to make a submission is modify the file <code>model.py</code> in the <code>sample_code_submission/</code> directory, then run this test to make sure everything works fine. This is the actual program that will be run on the server to test your submission. 

In [ ]:
outdir = 'sample_result_submission'     

In [ ]:
!python $problem_dir/ingestion.py $datadir $outdir $problem_dir $model_dir

## Preparing the submission

Zip the contents of `sample_code_submission/` (without the directory), or download the challenge public_data and run the command in the previous cell, after replacing sample_data by public_data.
Then zip the contents of `sample_result_submission/` (without the directory).
<b><span style="color:red">Do NOT zip the data with your submissions</span></b>.